In [101]:
import collections.abc
import pandas as pd
from datetime import datetime
import requests
import sys
from requests.auth import HTTPBasicAuth



In [152]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def correct_values(d):
    new_d = dict()
    mlsec = repr(d['values'][0][0]).split('.')[1]

    new_d['timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S.{}'.format(mlsec), time.gmtime(d['values'][0][0]))
    new_d['t2'] = d['values'][0][0]
    new_d['value'] = d['values'][0][1]
    del d['values']
    new_d['labels'] = d
    return new_d

In [153]:
import csv
import requests
import sys
from requests.auth import HTTPBasicAuth
def GetMetrixNames(url):
    new_url = '{0}/api/v1/label/__name__/values'.format(url)
    response = requests.get(new_url,auth=HTTPBasicAuth('admin', 'foobar'))
    names = response.json()['data']
    #Return metrix names
    return names
"""
Prometheus hourly data as csv.
"""

writer = csv.writer(open("metrics.csv",'w'))

metrixNames=GetMetrixNames("http://localhost:9090")
writeHeader=True
df = pd.DataFrame()

for metrixName in metrixNames:
     #now its hardcoded for hourly
    response = requests.get('{0}/api/v1/query'.format("http://localhost:9090"), params={'query': metrixName+'[1s]'},auth=HTTPBasicAuth('admin', 'foobar'))
    results = response.json()['data']['result']
    results = [correct_values(flatten(result)) for result in results] 
    df = df.append(pd.DataFrame().from_dict(results))

In [154]:
df

,timestamp,t2,value,labels
0,2020-09-23 21:52:49.152,1.600898e+09,1,"{'metric___name__': 'cadvisor_version_info', '..."
0,2020-09-23 21:52:49.297,1.600898e+09,8503,"{'metric___name__': 'classes', 'metric_instanc..."
1,2020-09-23 21:52:49.771,1.600898e+09,8659,"{'metric___name__': 'classes', 'metric_instanc..."
2,2020-09-23 21:52:49.223,1.600898e+09,7825,"{'metric___name__': 'classes', 'metric_instanc..."
0,2020-09-23 21:52:49.297,1.600898e+09,8503,"{'metric___name__': 'classes_loaded', 'metric_..."
...,...,...,...,...
10,2020-09-23 21:52:52.223,1.600898e+09,1,"{'metric___name__': 'up', 'metric_instance': '..."
11,2020-09-23 21:52:52.002,1.600898e+09,1,"{'metric___name__': 'up', 'metric_instance': '..."
0,2020-09-23 21:52:52.297,1.600898e+09,2230586,"{'metric___name__': 'uptime', 'metric_instance..."
1,2020-09-23 21:52:51.771,1.600898e+09,2228670,"{'metric___name__': 'uptime', 'metric_instance..."


In [151]:
datetime.strftime(df.iloc[0]['t2'])

TypeError: descriptor 'strftime' for 'datetime.date' objects doesn't apply to a 'numpy.float64' object

In [131]:
class TraceCollector:
    def __init__(self, url, username, password, output_file, from_date=None, to_date=None):
        self.username = username
        self.password = password
        self.url = url
        self.output_file = output_file
        self.from_date = from_date
        self.to_date = to_date
        
    def get_matrix_names(self, url)
        new_url = '{0}/api/v1/label/__name__/values'.format(url)
        response = requests.get(new_url,auth=HTTPBasicAuth(self.username, self.password))
        names = response.json()['data']
        return names
    
    def get_data(self,from_date=None,to_date=None):
        matrix_names = get_matrix_names(self.url)
        df = pd.DataFrame()
        for metrixName in metrixNames:
            response = requests.get('{0}/api/v1/query'.format("http://localhost:9090"), params={'query': metrixName+'[1s]'},auth=HTTPBasicAuth('admin', 'foobar'))
            results = response.json()['data']['result']
            results = [correct_values(flatten(result)) for result in results] 
            df = df.append(pd.DataFrame().from_dict(results))
            

        
    def flatten(d, parent_key='', sep='_'):
        items = []
        for k, v in d.items():
            new_key = parent_key + sep + k if parent_key else k
            if isinstance(v, collections.MutableMapping):
                items.extend(flatten(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)

    def correct_values(d):
        new_d = dict()
        new_d['timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(d['values'][0][0]))
        new_d['value'] = d['values'][0][1]
        del d['values']
        new_d['labels'] = d
        return new_d

In [117]:
df= df.set_index('timestamp')


In [123]:
df.index = pd.to_datetime(df.index)
df = df.sort_index()

,value,labels
timestamp,,
2020-09-23 19:21:48,1,"{'metric___name__': 'cadvisor_version_info', '..."
2020-09-23 19:21:48,10,"{'metric___name__': 'go_goroutines', 'metric_i..."
2020-09-23 19:21:48,0.292288033,{'metric___name__': 'go_gc_duration_seconds_su...
2020-09-23 19:21:48,0.571971612,{'metric___name__': 'go_gc_duration_seconds_su...
2020-09-23 19:21:48,0.238634491,{'metric___name__': 'go_gc_duration_seconds_su...
...,...,...
2020-09-23 19:21:51,51,"{'metric___name__': 'scrape_samples_scraped', ..."
2020-09-23 19:21:51,42,"{'metric___name__': 'scrape_samples_scraped', ..."
2020-09-23 19:21:51,0,"{'metric___name__': 'scrape_series_added', 'me..."


In [105]:
pd.DataFrame().from_records(results)

,timestamp,value,labels
0,2020-09-23 19:21:50,2386928,"{'metric___name__': 'uptime', 'metric_instance..."
1,2020-09-23 19:21:50,2387141,"{'metric___name__': 'uptime', 'metric_instance..."
2,2020-09-23 19:21:50,2386682,"{'metric___name__': 'uptime', 'metric_instance..."


In [111]:
datetime.now()

datetime.datetime(2020, 9, 23, 21, 38, 17, 95618)